In [22]:
class Sentiment:
    POSITIVE = 'POSITIVE'
    NEGATIVE = 'NEGATIVE'
    NEUTRAL = 'NEUTRAL'



class Review:
    def __init__(self,text,rating):
        self.text=text
        self.rating=rating
        self.sentiment=self.get_sentiment()


    def get_sentiment(self):
        if self.rating <=2:
            return Sentiment.NEGATIVE
        elif self.rating == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE

In [ ]:
import json
file_name = 'Books_small_10000.json'

# Update the get_sentiment method in the Review class

reviews=[]
with open(file_name) as f:
    for line in f:
       review = json.loads(line)
       reviews.append(Review(review['reviewText'], review['overall']))
    
reviews[10].sentiment


'POSITIVE'

prep Data

In [24]:
from sklearn.model_selection import train_test_split
training,test = train_test_split(reviews,test_size=0.33,random_state=42)  # 33% of the reviews will be for testing



In [34]:
# Extracting the text and rating  of each review from the training dataset
train_x = [x.text for x in training]

train_y = [x.sentiment for x in training]


test_x = [x.text for x in test]
test_y = [x.text for x in test]

train_x[7]
train_y[7]



'NEGATIVE'

bag of factroziation

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

# Extracting the text of each review from the test dataset


vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)
print(train_x_vectors[0].toarray())

[[0 0 0 ... 0 0 0]]


Classifiaction

In [37]:
#linear svm

from sklearn import svm

clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)

# Predicting the sentiment of the first test review
prediction = clf_svm.predict(test_x_vectors[0])

# Displaying the sentiment as POSITIVE, NEGATIVE, or NEUTRAL
print(prediction[0])  # This should display the sentiment label (e.g., POSITIVE, NEGATIVE, NEUTRAL)


POSITIVE


decision tree

In [40]:
from sklearn.tree import DecisionTreeClassifier
clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)
clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

naive bayes

In [57]:
from sklearn.naive_bayes import GaussianNB
clf_gnb =GaussianNB()
clf_gnb.fit(train_x_vectors.toarray(), train_y)
clf_gnb.predict(test_x_vectors[3].toarray())

array(['NEUTRAL'], dtype='<U8')

logistic regression

In [47]:
from sklearn.linear_model import LogisticRegression
clf_log=LogisticRegression()
clf_log.fit(train_x_vectors.toarray(), train_y)
clf_log.predict(test_x_vectors[0].toarray())

array(['POSITIVE'], dtype='<U8')

evalution

In [ ]:
# Ensure test_y contains the sentiments of the test reviews
test_y = [x.sentiment for x in test]
#mean accuracy
# Calculate and display the accuracy scores for SVM and Decision Tree classifiers
svm_score = clf_svm.score(test_x_vectors, test_y)
dec_score = clf_dec.score(test_x_vectors, test_y)

print(f"SVM Accuracy: {svm_score}")
print(f"Decision Tree Accuracy: {dec_score}")


SVM Accuracy: 0.8242424242424242
Decision Tree Accuracy: 0.7393939393939394


improving the model

In [60]:
train_y.count(Sentiment.NEGATIVE)

47